### Домашнее задание к занятию "Классификация: Логистическая регрессия и SVM"

Имеются данные adult.csv (см. в материалах к занятию).  
Целевой переменной является уровень дохода income (крайний правый столбец).  
Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html  
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.  
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.  


In [1]:
import pandas as pd

In [2]:
# В датасете пропущенные значения заменены заками вопроса. При чтении из файла вставим Nan вместо '?'
data = pd.read_csv('adult.csv', na_values='?')
data.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,NaN,227026,HS-grad,9,Never-married,NaN,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [4]:
# Посмотрим, сколько в датасете пропусков в данных
data.isna().sum()

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64

В данном случае пропуски составляют незначительную часть данных. Удалим строки, которые содержат пропуски

In [5]:
data.dropna(inplace=True)

In [6]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


Колонки 'workclass', 'education', 'marital-status', 'occupation','relationship', 'race', 'gender', 'native-country', 'income' содержат категориальные данные, но в каждом случае уникальных значений не очень много. Переведем данные в этих столбцах к числовому типу, используя LabelEncoder 

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
for col in data.select_dtypes(include='object').columns:
    data[col] = le.fit_transform(data[col].astype('str'))
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,2,226802,1,7,4,6,3,2,1,0,0,40,38,0
1,38,2,89814,11,9,2,4,0,4,1,0,0,50,38,0
2,28,1,336951,7,12,2,10,0,4,1,0,0,40,38,1
3,44,2,160323,15,10,2,6,0,2,1,7688,0,40,38,1
5,34,2,198693,0,6,4,7,1,4,1,0,0,30,38,0


In [10]:
# Разделим данные на целеую переменную и признаки
X = data.drop('income', axis=1)
Y = data['income']

**Строим модель линейной регрессии**

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# разделим выборку на обучающую и тестовую
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=146)

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
lr = LogisticRegression(max_iter=1000)

In [15]:
# обучим модель на тренировочных данных 
lr.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [16]:
print('Train score:', lr.score(x_train, y_train))
print('Test score:', lr.score(x_test, y_test))

Train score: 0.7989444509965797
Test score: 0.7937378383159385


Посмотрим, улучшится ли точность модели, если даные предварительно нормализовать.

In [17]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipe_model = make_pipeline(StandardScaler(),LogisticRegression(max_iter=1000))

In [18]:
pipe_model.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=1000))])

In [19]:
print('Train score:', pipe_model.score(x_train, y_train))
print('Test score:', pipe_model.score(x_test, y_test))

Train score: 0.8218539922160633
Test score: 0.8162037856005661


На нормализованных данных модель показала лучший результат.

**Строим модель SVM**  
Сразу используем StandartScaler() для нормализации данных

In [21]:
from sklearn.svm import SVC

Попробуем использовать модель с параметрами по умолчанию.

In [25]:
clf = make_pipeline(StandardScaler(), SVC())
clf.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])

In [26]:
print('Train score:', clf.score(x_train, y_train))
print('Test score:', clf.score(x_test, y_test))

Train score: 0.8538153084090105
Test score: 0.8428268176189634


При одном и том же разделении выборки на обучающую и тестовую модель SVM показала лучший score, чем модель логистической регрессии.  

Можно попробовать изменить kernel для SVM и посмотреть, даст ли это лучший результат. 
По умолчанию в модели используется параметр kernel='rbf', поэтому посмотрим на 'linear', 'poly' и 'sigmoid'. 
Для полиномиального ядра можно так же попробовать варьировать параметр degree - степень полиномиальной функции. 

In [36]:
for kernel in ['linear', 'poly', 'sigmoid']:
    if kernel == 'poly':
        for degree in range(1, 7):
            clf_new = make_pipeline(StandardScaler(), SVC(kernel=kernel, degree=degree))
            clf_new.fit(x_train, y_train)
            print(f'Kernel {kernel} degree {degree} train score: {clf_new.score(x_train, y_train)}')
            print(f'Kernel {kernel} degree {degree} test score: {clf_new.score(x_test, y_test)}')
            print()
    else:
        clf_new = make_pipeline(StandardScaler(), SVC(kernel=kernel, gamma='auto'))
        clf_new.fit(x_train, y_train)
        print(f'Kernel {kernel} train score: {clf_new.score(x_train, y_train)}')
        print(f'Kernel {kernel} test score: {clf_new.score(x_test, y_test)}')
        print()
    
    

Kernel linear train score: 0.809529425639816
Kernel linear test score: 0.8045285689014683

Kernel poly degree 1 train score: 0.8089987026772025
Kernel poly degree 1 test score: 0.8042632230673978

Kernel poly degree 2 train score: 0.8161339780634509
Kernel poly degree 2 test score: 0.8085087564125243

Kernel poly degree 3 train score: 0.8445866257813421
Kernel poly degree 3 test score: 0.8380505926056961

Kernel poly degree 4 train score: 0.8420509494044109
Kernel poly degree 4 test score: 0.8201839731116222

Kernel poly degree 5 train score: 0.8457954947517395
Kernel poly degree 5 test score: 0.8168229258800637

Kernel poly degree 6 train score: 0.8376872272673664
Kernel poly degree 6 test score: 0.8000176897222714

Kernel sigmoid train score: 0.75418681448284
Kernel sigmoid test score: 0.7441181673447727



Видим, что улучшения результата не произошло. Т.е. в нашем случае оптимально оставить использование ядра по умолчанию 'rbf'. Однако, если сравнивать с логистической регрессией, то SVM с полиномиальным ядром и степенью от 3 до 5 так же показывает большую точность. 